<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Build-a-simple-model" data-toc-modified-id="Build-a-simple-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Build a simple model</a></span><ul class="toc-item"><li><span><a href="#EX-1.-One-reaction-model" data-toc-modified-id="EX-1.-One-reaction-model-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><strong>EX 1</strong>. One reaction model</a></span></li></ul></li><li><span><a href="#Calculating-growth-rates" data-toc-modified-id="Calculating-growth-rates-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Calculating growth rates</a></span><ul class="toc-item"><li><span><a href="#The-Biomass-growth-objective-function" data-toc-modified-id="The-Biomass-growth-objective-function-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>The Biomass growth objective function</a></span></li><li><span><a href="#EX-2.-Growth-under-aerobic-conditions" data-toc-modified-id="EX-2.-Growth-under-aerobic-conditions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span><strong>EX 2</strong>. Growth under aerobic conditions</a></span></li><li><span><a href="#Map-Draw-Function" data-toc-modified-id="Map-Draw-Function-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Map Draw Function</a></span></li><li><span><a href="#EX-3.-Growth-in-ANAEROBIC-conditions" data-toc-modified-id="EX-3.-Growth-in-ANAEROBIC-conditions-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span><strong>EX 3</strong>. Growth in ANAEROBIC conditions</a></span></li><li><span><a href="#EX-4.-Growth-on-succinate" data-toc-modified-id="EX-4.-Growth-on-succinate-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span><strong>EX 4</strong>. Growth on succinate</a></span></li><li><span><a href="#EX-5.-Growth-on-succinate-under-anaerobic-conditions" data-toc-modified-id="EX-5.-Growth-on-succinate-under-anaerobic-conditions-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span><strong>EX 5</strong>. Growth on succinate under anaerobic conditions</a></span></li></ul></li></ul></div>

# Quantitative and Computational Biology
# Getting-Started

In [11]:
# Import Python Packages
import json # for saving files in json format
import cobra # main package
from escher import Builder # to draw maps
import pandas # to create dataframes and graphs
# to have inline matplotlib graphs
%matplotlib inline 

##  Build a simple model

In [12]:
from cobra import Model, Reaction, Metabolite
# Initialize Model
model = Model('simple_model')

In [14]:
# Create reaction (not yet in the model)
reaction = Reaction('R00658')
reaction.name = '2-phospho-D-glycerate hydro-lyase'
reaction.subsystem = 'Glycolysis (Embden-Meyerhof pathway)'
reaction.lower_bound = 0.0
reaction.upper_bound = 1000.0

simple_model


In [4]:
# Create Metabolites
pg2_c = Metabolite('2pg_c', 
                   formula='C3H7O7P', 
                   name='2-Phospho-D-glycerate', 
                   compartment='c')
pep_c = Metabolite('pep_c', 
                   formula='C3H5O6P', 
                   name='Phosphoenolpyruvate', 
                   compartment='c')
h2O_c = Metabolite('h2O_c', 
                   formula='H2O', 
                   name='H20', 
                   compartment='c')

In [5]:
reaction.add_metabolites({
    pg2_c: -1.0,
    pep_c: 1.0,
    h2O_c: 1.0
})

reaction.annotation['EC_number'] = '4.2.1.11'
#reaction.bounds = (-1000.0, 1000.0)
reaction.gene_reaction_rule = 'CP006584'
print(reaction.genes)
print(reaction)

frozenset({<Gene CP006584 at 0x7f2f7137d400>})
R00658: 2pg_c --> h2O_c + pep_c


In [7]:
model.add_reaction(reaction)
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

R00658 : 2pg_c --> h2O_c + pep_c
    2pg_c : C3H7O7P
    pep_c : C3H5O6P
    h2O_c : H2O
CP006584 is associated with reactions: {R00658}


In [15]:
print("Number of Reactions: %s" % len(model.reactions))
print("Number of Metabolites: {0}".format(len(model.metabolites)))
print(f"Number of Genes: {len(model.genes)}")

Number of Reactions: 0
Number of Metabolites: 0
Number of Genes: 0
simple_model


In [7]:
model.objective = 'R00658'
print(model.objective.expression)
print(model.objective.direction)

1.0*R00658 - 1.0*R00658_reverse_2bd5c
max


In [8]:
solution = model.optimize()
solution.fluxes

R00658    0.0
Name: fluxes, dtype: float64

### Model validation

In [9]:
from cobra.io import validate_sbml_model

In [10]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

(<Model simple_model at 0x120c93280>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


In [11]:
# INSERT HERE YOUR CODE

## Calculating growth rates

In [12]:
# load model (you can download it here: http://bigg.ucsd.edu/models/e_coli_core)
ecoli_core = cobra.io.read_sbml_model('./e_coli_core.xml')

# Create a copy of the original model (it will turn to be useful during the tutorial to reset the model)
ecoli_core_model = ecoli_core.copy()

# In the following three different ways of formatting output (more information here: https://pyformat.info/)
print("Number of Reactions: %s" % len(ecoli_core_model.reactions))
print("Number of Metabolites: {0}".format(len(ecoli_core_model.metabolites)))
print(f"Number of Genes: {len(ecoli_core_model.genes)}")

# NEW OBJECT FOR MEDIUM
print(ecoli_core_model.medium)
ecoli_core.reactions.ENO.reaction

Number of Reactions: 95
Number of Metabolites: 72
Number of Genes: 137
{'EX_co2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0}


'2pg_c <=> h2o_c + pep_c'

### The Biomass growth objective function 
The metabolic network includes a biomass production pseudo-reaction that consumes essential biomass metabolites, with its stoichiometric coefficients representing the molar quantities required for generating a unit mass of cells (in units of $mmol/gDW$). This reaction's flux activity represents the growth rate (in units of $1/h$).
source (Orth et al. 2010)


In [13]:
# Print objective function
print("Objective Function --> %s" % ecoli_core_model.objective)
obj_fnc = ecoli_core_model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM")
print(obj_fnc.reaction)

Objective Function --> Maximize
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c


### **EX 2**. Growth under aerobic conditions

In [14]:
# Change reaction EX_glc(e) lower bound. the upper limit of glucose uptake rate is set by default to 
# 10 mmol/g-dry cell/hr
# By convention, exchange reactions are written as export reactions (e.g. EX_glc__D_e <==> glc__D_e),
# so import of a metabolite is a negative flux
ecoli_core_model.reactions.get_by_id('EX_glc__D_e').lower_bound = -18.5
ecoli_core_model.reactions.get_by_id('EX_o2_e').lower_bound = -1000
lb, ub = ecoli_core_model.reactions.get_by_id('EX_glc__D_e').bounds
print("Glucose Uptake bounds: {0} < v < {1}".format(lb, ub))
lb, ub = ecoli_core_model.reactions.get_by_id('EX_o2_e').bounds
print("Oxigen Uptake bounds: {0} < v < {1}".format(lb, ub))
# Optimize model 
solution = ecoli_core_model.optimize()
# Parsimoniuous FBA (minimize total sum of fluxes producing approximately the same result, see below)
solution_pFBA = cobra.flux_analysis.pfba(ecoli_core_model)
# show FBA results
print("Objective function solution: %5f" % solution.objective_value) # Objective function 
print("Linear Solver Status: %s" % solution.status) # status of the linear prog solver
print(ecoli_core_model.summary())
print(abs(solution.fluxes["BIOMASS_Ecoli_core_w_GAM"]-solution_pFBA.fluxes["BIOMASS_Ecoli_core_w_GAM"]))
display(ecoli_core_model.medium)

Glucose Uptake bounds: -18.5 < v < 1000.0
Oxigen Uptake bounds: -1000 < v < 1000.0
Objective function solution: 1.653072
Linear Solver Status: optimal
Objective
1.0 BIOMASS_Ecoli_core_w_GAM = 1.6530718511568285

Uptake
------
Metabolite    Reaction  Flux  C-Number  C-Flux
  glc__D_e EX_glc__D_e  18.5         6 100.00%
     nh4_e    EX_nh4_e 9.014         0   0.00%
      o2_e     EX_o2_e 38.74         0   0.00%
      pi_e     EX_pi_e 6.081         0   0.00%

Secretion
---------
Metabolite Reaction   Flux  C-Number  C-Flux
     co2_e EX_co2_e -40.65         1 100.00%
     h2o_e EX_h2o_e -52.69         0   0.00%
       h_e   EX_h_e -33.16         0   0.00%

8.881784197001252e-16


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 18.5,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000,
 'EX_pi_e': 1000.0}

In [15]:
ecoli_core_model.metabolites.atp_c.summary()

Percent,Flux,Reaction,Definition
66.48%,80.61,ATPS4r,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c
24.18%,29.31,PGK,3pg_c + atp_c <=> 13dpg_c + adp_c
2.26%,2.744,PYK,adp_c + h_c + pep_c --> atp_c + pyr_c
7.08%,8.582,SUCOAS,atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c
Percent,Flux,Reaction,Definition
6.92%,-8.39,ATPM,atp_c + h2o_c --> adp_c + h_c + pi_c
81.55%,-98.87,BIOMASS_Ecoli_core_w_GAM,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
0.35%,-0.4227,GLNS,atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c
11.19%,-13.56,PFK,atp_c + f6p_c --> adp_c + fdp_c + h_c


**Here is high flux in the glycolysis, pentose phosphate, TCA cycle, and oxidative phosphorylation pathways, and that no organic by-products are secreted**

In [16]:
import escher
import escher.urls
from escher import Builder
# save model in JSON to communicate with escher viewer and create the new map
# TO UPLOAD AND VISUALIZE RESULTS ON ESCHER (https://escher.github.io/).
cobra.io.save_json_model(ecoli_core_model, 'ecoli_core_model.json')
with open('ecoli_core_EX_glc__D_e_reactions.json', 'w') as outfile:  
    json.dump(solution.fluxes.to_dict(), outfile)
with open('ecoli_core_EX_glc__D_e_metabolites.json', 'w') as outfile:  
    json.dump(solution.shadow_prices.to_dict(), outfile)
# END JSON CREATION

# save model in JSON to communicate with escher viewer and create the new map
# TO WRITE JSON --> cobra.io.write_sbml_model(emodel, "model.xml")
cobra.io.save_json_model(ecoli_core_model, 'ecoli_core_model_ver1.json')

with open('ecoli_core_EX_glc__D_e_reactions_ver1.json', 'w') as outfile:  
    json.dump(solution.fluxes.to_dict(), outfile)

with open('ecoli_core_EX_glc__D_e_metabolites_ver1.json', 'w') as outfile:  
    json.dump(solution.shadow_prices.to_dict(), outfile)

# Create the builder for the model
b = Builder(map_json="./e_coli_core.Core metabolism.json",
            model=ecoli_core_model,
            reaction_data=solution.fluxes.to_dict(),
            reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
            metabolite_data=solution.shadow_prices.to_dict(),
            hide_secondary_metabolites=False)
b


Builder(hide_secondary_metabolites=False, metabolite_data={'13dpg_c': -0.047105494664984385, '2pg_c': -0.04201…

**RESULTS**. There is high flux in the glycolysis, pentose phosphate, TCA cycle, and oxidative phosphorylation pathways, and that no organic by-products are secreted

### Map Draw Function
Define a function to draw maps every time you need all over the tutorial. 

Map downloaded from https://escher.github.io/ (LOAD MAP -> MAP -> SAVE MAP (JSON))

In [17]:
def draw_map(model, solution):
    b = Builder(map_json="./e_coli_core.Core metabolism.json", # Change with the path on your PC. 
            model=model,
            reaction_data=solution.fluxes.to_dict(),
            reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
            metabolite_data=solution.shadow_prices.to_dict(),
            hide_secondary_metabolites=False)

    return b

### **EX 3**. Growth in ANAEROBIC conditions

In [18]:
# reset model
ecoli_core_model = ecoli_core.copy()
ecoli_core_model.reactions.get_by_id('EX_glc__D_e').lower_bound = -18.5
lb, ub = ecoli_core_model.reactions.get_by_id('EX_o2_e').bounds
print("Oxigen Uptake bounds: {0} < v < {1}".format(lb, ub))
ecoli_core_model.reactions.get_by_id('EX_o2_e').lower_bound = 0.0
lb, ub = ecoli_core_model.reactions.get_by_id('EX_o2_e').bounds
print("Oxigen Uptake bounds: {0} < v < {1}".format(lb, ub))
# Optimize model 
solution = ecoli_core_model.optimize()
# show FBA results
print("Objective function solution: %5f" % solution.objective_value) # Objective function 
print("Linear Solver Status: %s" % solution.status) # status of the linear prog solver
ecoli_core_model.summary()
ecoli_core_model.medium

Oxigen Uptake bounds: -1000.0 < v < 1000.0
Oxigen Uptake bounds: 0.0 < v < 1000.0
Objective function solution: 0.470565
Linear Solver Status: optimal


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 18.5,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

In [19]:
draw_map(ecoli_core_model, solution)

Builder(hide_secondary_metabolites=False, metabolite_data={'13dpg_c': -0.03599710029512122, '2pg_c': -0.024921…

**The flux distribution shows that oxidative phosphorylation is not used in these conditions, and that acetate, formate, and ethanol are produced by fermentation pathways**

### **EX 4**. Growth on succinate 
The core E. coli model contains exchange reactions for 13 different organic compounds, each of which can be used as carbon source under both aerobic anaerobic conditions.

In [20]:
# Carbon Source Reaction IDs
EX_uptakes = [
    'EX_ac_e',
    'EX_acald_e',
    'EX_akg_e',
    'EX_etoh_e',
    'EX_for_e', 
    'EX_fru_e',
    'EX_fum_e',
    'EX_gln__L_e', 
    'EX_glu__L_e',
    'EX_glc__D_e',
    'EX_lac__D_e', 
    'EX_mal__L_e',
    'EX_pyr_e',
    'EX_succ_e',
]
for i in EX_uptakes:
    rct = ecoli_core_model.reactions.get_by_id(i)
    print('%s %s [%s]: %s' % (rct.id, rct.bounds, rct.name, rct.reaction))
    
ecoli_core_model.medium

EX_ac_e (0.0, 1000.0) [Acetate exchange]: ac_e --> 
EX_acald_e (0.0, 1000.0) [Acetaldehyde exchange]: acald_e --> 
EX_akg_e (0.0, 1000.0) [2-Oxoglutarate exchange]: akg_e --> 
EX_etoh_e (0.0, 1000.0) [Ethanol exchange]: etoh_e --> 
EX_for_e (0.0, 1000.0) [Formate exchange]: for_e --> 
EX_fru_e (0.0, 1000.0) [D-Fructose exchange]: fru_e --> 
EX_fum_e (0.0, 1000.0) [Fumarate exchange]: fum_e --> 
EX_gln__L_e (0.0, 1000.0) [L-Glutamine exchange]: gln__L_e --> 
EX_glu__L_e (0.0, 1000.0) [L-Glutamate exchange]: glu__L_e --> 
EX_glc__D_e (-18.5, 1000.0) [D-Glucose exchange]: glc__D_e <=> 
EX_lac__D_e (0.0, 1000.0) [D-lactate exchange]: lac__D_e --> 
EX_mal__L_e (0.0, 1000.0) [L-Malate exchange]: mal__L_e --> 
EX_pyr_e (0.0, 1000.0) [Pyruvate exchange]: pyr_e --> 
EX_succ_e (0.0, 1000.0) [Succinate exchange]: succ_e --> 


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 18.5,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

In [21]:
# reset model
ecoli_core_model = ecoli_core.copy()
# Change substrate to EX_succ_e
ecoli_core_model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0.0
lb, ub = ecoli_core_model.reactions.get_by_id('EX_glc__D_e').bounds
print("Glucose Uptake bounds: {0} < v < {1}".format(lb, ub))
ecoli_core_model.reactions.get_by_id('EX_succ_e').lower_bound = -18.0
lb, ub = ecoli_core_model.reactions.get_by_id('EX_succ_e').bounds
print("Succinate Uptake bounds: {0} < v < {1}".format(lb, ub))


Glucose Uptake bounds: 0.0 < v < 1000.0
Succinate Uptake bounds: -18.0 < v < 1000.0


In [22]:
# Optimize model 
solution_succ = ecoli_core_model.optimize()
# show FBA results
print("Objective function solution: %5f" % solution_succ.objective_value) # Objective function 
print("Linear Solver Status: %s" % solution_succ.status) # status of the linear prog solver
ecoli_core_model.medium
ecoli_core_model.summary()

Objective function solution: 0.751620
Linear Solver Status: optimal


Metabolite,Reaction,Flux,C-Number,C-Flux
h_e,EX_h_e,20.92,0,0.00%
nh4_e,EX_nh4_e,4.098,0,0.00%
o2_e,EX_o2_e,30.15,0,0.00%
pi_e,EX_pi_e,2.765,0,0.00%
succ_e,EX_succ_e,18,4,100.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-40.01,1,100.00%
h2o_e,EX_h2o_e,-27.49,0,0.00%


In [23]:
draw_map(ecoli_core_model, solution)

Builder(hide_secondary_metabolites=False, metabolite_data={'13dpg_c': -0.03599710029512122, '2pg_c': -0.024921…

### **EX 5**. Growth on succinate under anaerobic conditions

In [24]:
# reset model

ecoli_core_model = ecoli_core.copy()
print(ecoli_core_model.medium)
# Stop Glucose uptake
ecoli_core_model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0.0
lb, ub = ecoli_core_model.reactions.get_by_id('EX_glc__D_e').bounds
print("Glucose Uptake bounds: {0} < v < {1}".format(lb, ub))

# Activate succinate uptake
ecoli_core_model.reactions.get_by_id('EX_succ_e').lower_bound = -18.0
lb, ub = ecoli_core_model.reactions.get_by_id('EX_succ_e').bounds
print("Succinate Uptake bounds: {0} < v < {1}".format(lb, ub))

# Go to anaerobic
ecoli_core_model.reactions.get_by_id('EX_o2_e').lower_bound = 0.0
lb, ub = ecoli_core_model.reactions.get_by_id('EX_o2_e').bounds
print("Oxigen Uptake bounds: {0} < v < {1}".format(lb, ub))
print(ecoli_core_model.medium)

{'EX_co2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0}
Glucose Uptake bounds: 0.0 < v < 1000.0
Succinate Uptake bounds: -18.0 < v < 1000.0
Oxigen Uptake bounds: 0.0 < v < 1000.0
{'EX_co2_e': 1000.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_succ_e': 18.0}


In [25]:
# Optimize model 
solution = ecoli_core_model.optimize()
# show FBA results
print("Linear Solver Status: %s" % solution.status) # status of the linear prog solver

Linear Solver Status: infeasible


/Users/afilisetti/Documents/VENVS/env-mge-22/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


FBA predicts that **growth is not possible on succinate under anaerobic conditions**. Because the maximum amount of ATP that can be produced from this amount of succinate is less than the minimum bound of **$8.39~mmol~gDW^{-1}~hr^{-1}$ of the ATP maintenance reaction (ATPM, check the value with aerobic environemnt), there is no feasible solution**, (Orth et al. 2010). FBA predicted growth rates for all 13 organic substrates in the E. coli core model. The growth rates are all much lower anaerobically (0 hr-1 in most cases) because the electron transport chain cannot be used to fully oxidize the substrates and generate as much ATP.

# References
The following tutorial has been made by integrating and adapting the information contained in the following references:

- Klipp, E., Liebermeister, W., Wierling, C., Kowald, A., Lehrach, H., & Herwig, R. (2009). Systems Biology: A Textbook. Time. https://doi.org/10.3797/scipharm
- Orth, J. D., Thiele, I., & Palsson, B. O. (2010). What is flux balance analysis? Nature Biotechnology, 28(3), 245–248. https://doi.org/10.1038/nbt.1614
- Thiele, I. and B. O. Palsson (2010). "A protocol for generating a high‐quality genome‐scale metabolic reconstruction." Nature protocols 5(1): 93‐121. 
- O’Brien, E. J., Monk, J. M., & Palsson, B. O. (2015). Using Genome-scale Models to Predict Biological Capabilities. Cell, 161(5), 971–987. https://doi.org/10.1016/j.cell.2015.05.019
- Monk, J., Nogales, J., & Palsson, B. O. (2014). Optimizing genome-scale network reconstructions. Nature Biotechnology, 32(5), 447–452. https://doi.org/10.1038/nbt.2870
- Harcombe, W. R., Riehl, W. J., Dukovski, I., Granger, B. R., Betts, A., Lang, A. H., … Segrè, D. (2014). Metabolic resource allocation in individual microbes determines ecosystem interactions and spatial dynamics. Cell Reports, 7(4), 1104–1115. https://doi.org/10.1016/j.celrep.2014.03.070
- Kim, M. W., Sang Yi, J., Kim, J.-N. J. J.-N. N. J. J. J., Kim, J.-N. J. J.-N. N. J. J. J., Kim, M. W., Kim, B.-G. G., … Kim, B.-G. G. (2014). Reconstruction of high-quality metabolic model enables identification of gene overexpression targets for enhanced antibiotics production in Streptomyces coelicolor A3(2). Biotechnology Journal, 9(9), 1–33. https://doi.org/10.1002/biot.201300539.Submitted
- O’Brien, E. J., Lerman, J. A., Chang, R. L., Hyduke, D. R., & Palsson, B. Ø. (2013). Genome-scale models of metabolism and gene expression extend and refine growth phenotype prediction. Molecular Systems Biology, 9, 693. https://doi.org/10.1038/msb.2013.52
- Reznik, E., Mehta, P., & Segr??, D. (2013). Flux Imbalance Analysis and the Sensitivity of Cellular Growth to Changes in Metabolite Pools. PLoS Computational Biology, 9(8). https://doi.org/10.1371/journal.pcbi.1003195
- Shimizu, K. (2013). Bacterial Cellular Metabolic Systems. Bacterial Cellular Metabolic Systems. https://doi.org/10.1533/9781908818201.95
- Manuscript, A. (2013). NIH Public Access. Appl Microbiol Biotechnol, 81(4), 291–305. https://doi.org/10.1038/nrmicro2737.Constraining
- Park, J. M., Park, H. M., Kim, W. J., Kim, H. U., Kim, T. Y., & Lee, S. Y. (2012). Flux variability scanning based on enforced objective flux for identifying gene amplification targets. BMC Systems Biology, 6, 106. https://doi.org/10.1186/1752-0509-6-106
- Blazier, A. S., & Papin, J. A. (2012). Integration of expression data in genome-scale metabolic network reconstructions. Frontiers in Physiology, 3, 299. https://doi.org/10.3389/fphys.2012.00299
- Noor, E., Lewis, N. E., & Milo, R. (2012). A proof for loop-law constraints in stoichiometric metabolic networks. BMC Systems Biology, 6, 140. https://doi.org/10.1186/1752-0509-6-140
Copeland, W. B., Bartley, B. A., Chandran, D., Galdzicki, M., Kim, K. H., Sleight, S. C., … Sauro, H. M. (2012). - - Computational tools for metabolic engineering. Metabolic Engineering, 14(3), 270–280. https://doi.org/10.1016/j.ymben.2012.03.001
- Lewis, N. E., Nagarajan, H., & Palsson, B. O. (2012). Constraining the metabolic genotype-phenotype relationship using a phylogeny of in silico methods. Nature Reviews. Microbiology, 10(4), 291–305. https://doi.org/10.1038/nrmicro2737
- Curran, K. A., Crook, N. C., & Alper, H. S. (2012). Using flux balance analysis to guide microbial metabolic engineering. Methods in Molecular Biology (Clifton, N.J.), 834, 197–216. https://doi.org/10.1007/978-1-61779-483-4_13
- Wessely, F., Bartl, M., Guthke, R., Li, P., Schuster, S., & Kaleta, C. (2011). Optimal regulatory strategies for metabolic pathways in Escherichia coli depending on protein costs. Molecular Systems Biology, 7, 515. https://doi.org/10.1038/msb.2011.46
- Chen, X., Alonso, A. P., Allen, D. K., Reed, J. L., & Shachar-Hill, Y. (2011). Synergy between 13C-metabolic flux analysis and flux balance analysis for understanding metabolic adaption to anaerobiosis in E. coli. Metabolic Engineering, 13(1), 38–48. https://doi.org/10.1016/j.ymben.2010.11.004
- Schellenberger, J., Que, R., Fleming, R. M. T., Thiele, I., Orth, J. D., Feist, A. M., … Palsson, B. Ø. (2011). Quantitative prediction of cellular metabolism with constraint-based models: the COBRA Toolbox v2.0. Nature Protocols, 6(9), 1290–1307. https://doi.org/10.1038/nprot.2011.308
- Toya, Y., Kono, N., Arakawa, K., & Tomita, M. (2011). Metabolic flux analysis and visualization. Journal of Proteome Research, 10(8), 3313–3323. https://doi.org/10.1021/pr2002885
- Zhang, W., Li, F., & Nie, L. (2010). Integrating multiple “omics” analysis for microbial biology: application and methodologies. Microbiology, 156(2), 287–301. https://doi.org/10.1099/mic.0.034793-0
- Orth, J. D., Palsson, B. Ø., Fleming, R. M. T., & Palsson, B. Ø. (2010). Reconstruction and Use of Microbial Metabolic Networks: the Core Escherichia coli Metabolic Model as an Educational Guide. EcoSal Plus, 4(1), Ch.10.2.1. https://doi.org/10.1128/ecosalplus.10.2.1
- Feist, A. M., & Palsson, B. O. (2010). The biomass objective function. Current Opinion in Microbiology. https://doi.org/10.1016/j.mib.2010.03.003
- Orth, J. D., Fleming, R. M. T., & Palsson, B. Ø. (2010). Reconstruction and Use of Microbial Metabolic Networks: the Core Escherichia coli Metabolic Model as an Educational Guide. EcoSal, (2), Ch.10.2.1. https://doi.org/10.1128/ecosal.10.2.1